Import Libraires

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import warnings ; warnings.filterwarnings('ignore')
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error

# fixed seed
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(SEED)

Load Data

In [15]:
path = 'data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

Drop unnecessary column

In [16]:
x_train = train.drop('ID', axis=1)
y_train = train['풍속 (m/s)']
x_test = test.drop('ID', axis=1)

LabelEncoder & OneHotEncoder

In [17]:
le = LabelEncoder()
le.fit(x_train['측정 시간대'])
x_train['측정 시간대'] = le.transform(x_train['측정 시간대'])
x_test['측정 시간대'] = le.transform(x_test['측정 시간대'])

In [18]:
temp = np.array(x_train['월']).reshape(-1, 1)
ohe = OneHotEncoder()
ohe.fit(temp)
temp = ohe.transform(temp)
temp = temp.toarray()
temp = pd.DataFrame(temp, columns=[i for i in range(1, 13)], dtype=int)
x_train = pd.concat([x_train.drop('월', axis=1), temp], axis=1)

temp = np.array(x_test['월']).reshape(-1, 1)
temp = ohe.transform(temp)
temp = temp.toarray()
temp = pd.DataFrame(temp, columns = [i for i in range(1, 13)], dtype=int)
x_test = pd.concat([x_test.drop('월', axis=1), temp], axis=1)


Autogluon

In [ ]:
train_data = TabularDataset(x_train)
test_data = TabularDataset(x_test)

save_path = '4th_autogluon/mae_5.5hours'
predictor = TabularPredictor(label = '풍속 (m/s)',
                             save_path = save_path,
                             eval_metric = 'mean_absolute_error',
                             ).fit(
                                 train_data = train_data,
                                 presets = 'best_quality',
                                 time_limit= 3600 * 5.5,
                                 num_stack_levels = 3
                             )
pred = predictor.predict(test_data)
sub = pd.read_csv(path + 'sample_submission.csv')
sub['풍속 (m/s)'] = pred
# sub.to_csv(path + '4th_submission_5.5hours.csv', index=False)

In [ ]:
train_data = TabularDataset(x_train)
test_data = TabularDataset(x_test)

save_path = '4th_autogluon/mae_3.5hours'
predictor = TabularPredictor(label = '풍속 (m/s)',
                             save_path = save_path,
                             eval_metric = 'mean_absolute_error',
                             ).fit(
                                 train_data = train_data,
                                 presets = 'best_quality',
                                 time_limit= 3600 * 3.5,
                                 num_stack_levels = 3
                             )
pred = predictor.predict(test_data)
sub = pd.read_csv(path + 'sample_submission.csv')
sub['풍속 (m/s)'] = pred
# sub.to_csv(path + '4th_submission_3.5hours.csv', index=False)

Ensemble

In [ ]:
df1 = pd.read_csv(path + '4th_submission_5.5hours.csv')
df2 = pd.read_csv(path + '4th_submission_3.5hours.csv')
ensembled = (df1['풍속 (m/s)'] + df2['풍속 (m/s)']) / 2
df2['풍속 (m/s)'] = ensembled
df2.to_csv(path + '4th_submission_ensembled.csv', index=False)